In [189]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [237]:
# school_set = set()
# for idx in tqdm(range(len(school))):
#     j  = school.at[idx,'SC4'].get('meta').get('total_count')
#     for jdx in range(j):
#         data = school.at[0,'SC4'].get('documents')[jdx].get('category_name').split('>')[2].strip(' ')
#         school_set.add(data)
target = 'PO3'


target_set = set()

temp_list =[]

for idx in tqdm(range(len(school))):
    sc4_data = school.at[idx, target]
    if isinstance(sc4_data, dict):  # Check if SC4 data is a dictionary
        j = sc4_data.get('meta', {}).get('total_count', 0)
        for jdx in range(j):
            documents = sc4_data.get('documents', [])
            if jdx < len(documents):
                data = documents[jdx].get('category_name', '').split('>')[2].strip(' ')
                # if data != '초등학교':
                #     위도 = float(documents[jdx].get('y'))
                #     경도 = float(documents[jdx].get('x'))
                #     학교이름 = documents[jdx].get('place_name')
                #     d1 ={'학교':data,'위도':위도,'경도':경도,'학교이름':학교이름}
                #     temp_list.append(d1)
                target_set.add(data)
    # school_df.at[idx,'school'] = temp_list

target_set


100%|██████████| 2314/2314 [00:00<00:00, 54767.79it/s]


{'검찰청', '경찰서', '구청', '법원', '소방서', '시,군,구민회관', '시청', '행정복지센터'}

In [238]:
from datetime import datetime
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 

bus_station = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/버스정류장/Bus_around_500_08231838.pickle')
# data = pd.read_pickle(r'C:\Users\Lee\Desktop\천안시_DataDir\버스데이터\Bus_around_500_08231838.pickle')
bus_station

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,0,0,0,0,0,8,0,1,0
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,0,3,0,0,0
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,0,0,0,0,0,0,7,0,0,0
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,0,0,0,0,0,6,205,0,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,GGB285000034,성환터미널,36.916583,127.132933,1,12,4,3,30,3,...,0,0,0,0,1,6,0,32,0,0
2310,GGB285000035,남서울대학,36.910267,127.142200,0,5,1,1,1,8,...,0,0,0,0,1,0,0,9,0,0
2311,GGB285500010,천안TG.단대.천안대,36.822650,127.162867,0,12,3,1,37,8,...,0,0,11,0,0,17,0,27,0,0
2312,GGB285500020,천안종합터미널,36.819833,127.155733,2,34,16,0,115,22,...,0,6,54,0,1,28,0,166,0,0


In [240]:
outer = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/버스정류장/Bus_Main0823.pickle')
cols = ['정류장번호'] + list(outer.columns[outer.columns.get_loc('공원거리구분_A'):])
outer_cp = outer[cols].copy()
outer_cp

,정류장번호,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,0,0,5,25,1,0
1,ASB285000078,0,0,11,15,0,1
2,ASB285000079,0,0,12,22,0,0
3,ASB285000094,0,0,5,24,1,0
4,ASB285000095,0,1,11,16,0,1
...,...,...,...,...,...,...,...
2309,GGB285000034,1,0,11,16,0,1
2310,GGB285000035,0,1,11,9,0,0
2311,GGB285500010,4,4,17,31,0,0
2312,GGB285500020,6,8,14,62,0,0


In [241]:
merge1 = pd.merge(bus_station,outer_cp,on='정류장번호',how='left')
len(data)==len(merge1)
merge1

target_list = list(merge1.columns[merge1.columns.get_loc('MT1'):merge1.columns.get_loc('PM9')+1])
target_list
merge1['sum']=merge1[target_list].sum(axis=1)
merge1

score_df = merge1.copy()
# score 칼럼 구하기
score_df['score'] = score_df['sum'].astype('int') + (score_df['공원거리구분_A']*2 + score_df['공원거리구분_B']) + (score_df['CH거리구분_A'] *2 + score_df['CH거리구분_B']) + (score_df['하천거리구분_A'] *2 + score_df['하천거리구분_B'])
# score_df[['주변시설의총합','score']]
sorted_score_df = score_df.sort_values(by='score',ascending=False)
sorted_score_df

# sorted_score_df =score_df[['정류장번호','정류장명','위도','경도','도로명주소','시','구','읍/면/동','리','score']].sort_values(by='score',ascending=False)
# sorted_score_df


False

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,FD6,CE7,HP8,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,8,0,1,0,0,0,5,25,1,0
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,3,0,0,0,0,0,11,15,0,1
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,0,0,0,12,22,0,0
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,7,0,0,0,0,0,5,24,1,0
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,205,0,16,0,0,1,11,16,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,GGB285000034,성환터미널,36.916583,127.132933,1,12,4,3,30,3,...,0,32,0,0,1,0,11,16,0,1
2310,GGB285000035,남서울대학,36.910267,127.142200,0,5,1,1,1,8,...,0,9,0,0,0,1,11,9,0,0
2311,GGB285500010,천안TG.단대.천안대,36.822650,127.162867,0,12,3,1,37,8,...,0,27,0,0,4,4,17,31,0,0
2312,GGB285500020,천안종합터미널,36.819833,127.155733,2,34,16,0,115,22,...,0,166,0,0,6,8,14,62,0,0


['MT1',
 'CS2',
 'PS3',
 'SC4',
 'AC5',
 'PK6',
 'OL7',
 'SW8',
 'BK9',
 'CT1',
 'AG2',
 'PO3',
 'AT4',
 'AD5',
 'FD6',
 'CE7',
 'HP8',
 'PM9']

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,CE7,HP8,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,sum
0,ASB285000077,성환8리,36.921468,127.122774,0,2,1,0,0,2,...,0,1,0,0,0,5,25,1,0,17
1,ASB285000078,송덕,36.923783,127.118976,0,2,2,1,0,2,...,0,0,0,0,0,11,15,0,1,12
2,ASB285000079,국립축산과학원,36.925931,127.111136,0,1,1,1,0,0,...,0,0,0,0,0,12,22,0,0,3
3,ASB285000094,성환8리,36.921479,127.122505,0,2,1,0,0,2,...,0,0,0,0,0,5,24,1,0,15
4,ASB285000095,성환터미널,36.917221,127.132732,1,11,4,3,30,3,...,0,16,0,0,1,11,16,0,1,280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,GGB285000034,성환터미널,36.916583,127.132933,1,12,4,3,30,3,...,32,0,0,1,0,11,16,0,1,92
2310,GGB285000035,남서울대학,36.910267,127.142200,0,5,1,1,1,8,...,9,0,0,0,1,11,9,0,0,26
2311,GGB285500010,천안TG.단대.천안대,36.822650,127.162867,0,12,3,1,37,8,...,27,0,0,4,4,17,31,0,0,116
2312,GGB285500020,천안종합터미널,36.819833,127.155733,2,34,16,0,115,22,...,166,0,0,6,8,14,62,0,0,444


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,HP8,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,sum,score
856,CAB285000936,성정중학교,36.830708,127.137961,1,66,1,1,25,27,...,32,0,5,10,4,47,0,0,1145,1220
174,ASB288001967,불당상업지구입구,36.811821,127.108607,1,27,11,0,270,36,...,74,0,2,7,0,3,1,0,1201,1217
857,CAB285000937,성정중학교,36.829957,127.137722,2,61,1,1,16,25,...,50,0,5,9,3,60,0,0,1098,1183
855,CAB285000935,두정양돈농협,36.833573,127.138038,1,54,9,3,70,19,...,21,0,5,7,7,28,0,0,1085,1144
924,CAB285001024,두정도서관,36.833743,127.135533,1,49,13,5,79,18,...,22,0,7,6,8,23,0,0,1069,1128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,CAB285071671,의관동종점,36.822265,127.326065,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017,CAB285071513,보산원1리,36.690781,127.074633,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
521,CAB285000499,덕전1리,36.809343,127.221424,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
524,CAB285000502,덕전1리(용소),36.808634,127.224933,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
# 겹쳐져 있는 버스정류장 (하나가 선택되었다면, 맞은 편의 버스정류장도 선택되기 때문에 (이름이 같으니)) 정류장명 중복 삭제
unique_sorted_score_df = sorted_score_df.drop_duplicates(subset='정류장명')

In [229]:
cutted_sorted_score_df = unique_sorted_score_df.head(300)
cutted_sorted_score_df

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,HP8,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,sum,score
856,CAB285000936,성정중학교,36.830708,127.137961,1,66,1,1,25,27,...,32,0,5,10,4,47,0,0,1145,1220
174,ASB288001967,불당상업지구입구,36.811821,127.108607,1,27,11,0,270,36,...,74,0,2,7,0,3,1,0,1201,1217
855,CAB285000935,두정양돈농협,36.833573,127.138038,1,54,9,3,70,19,...,21,0,5,7,7,28,0,0,1085,1144
924,CAB285001024,두정도서관,36.833743,127.135533,1,49,13,5,79,18,...,22,0,7,6,8,23,0,0,1069,1128
904,CAB285000995,벽산블루밍아파트,36.826201,127.113307,0,8,15,1,10,2,...,1,0,1,10,430,114,0,0,131,1117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,CAB285071775,쌍용역(방아다리공원방면),36.793798,127.120947,0,21,25,2,43,28,...,0,0,4,14,10,28,0,0,160,230
2060,CAB285071558,현대6차아파트,36.791416,127.124305,1,19,23,3,75,15,...,0,0,3,12,10,36,0,0,153,227
180,ASB288010485,Y시티아파트,36.797603,127.108500,1,13,2,1,7,17,...,14,0,1,6,0,9,0,0,210,227
1970,CAB285071457,제5산단,36.740141,127.255961,0,1,0,0,0,0,...,0,0,2,2,83,51,0,1,1,225


# 클러스터링

In [230]:
from sklearn.cluster import KMeans

data = cutted_sorted_score_df.copy()
coordinates = data[['위도', '경도']]
num_clusters = 10 # You can change this number
kmeans = KMeans(n_clusters=num_clusters)

# Fit the model to the coordinates
kmeans.fit(coordinates)

# Add a new column to your DataFrame with the cluster labels
data['cluster'] = kmeans.labels_

# Print the cluster centers (the centroids)

'''
센트럴 포인트에서 가까운 정류장만 고려하기?
'''
print("Cluster Centers:")
print(kmeans.cluster_centers_)
print(data['cluster'].value_counts())

# Print the first few rows of the DataFrame with the cluster labels
data.head()

KMeans(n_clusters=10)

'\n센트럴 포인트에서 가까운 정류장만 고려하기?\n'

Cluster Centers:
[[ 36.82710567 127.13806741]
 [ 36.86264805 127.11416226]
 [ 36.79567931 127.15692356]
 [ 36.80416002 127.11243248]
 [ 36.79717066 127.13020216]
 [ 36.72900452 127.1006628 ]
 [ 36.90228036 127.14513025]
 [ 36.82417254 127.11161804]
 [ 36.7401407  127.2559612 ]
 [ 36.81386361 127.15025859]]
0    59
4    58
3    43
9    43
7    41
2    26
1    14
6    10
5     5
8     1
Name: cluster, dtype: int64


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PM9,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,sum,score,cluster
856,CAB285000936,성정중학교,36.830708,127.137961,1,66,1,1,25,27,...,0,5,10,4,47,0,0,1145,1220,0
174,ASB288001967,불당상업지구입구,36.811821,127.108607,1,27,11,0,270,36,...,0,2,7,0,3,1,0,1201,1217,3
855,CAB285000935,두정양돈농협,36.833573,127.138038,1,54,9,3,70,19,...,0,5,7,7,28,0,0,1085,1144,0
924,CAB285001024,두정도서관,36.833743,127.135533,1,49,13,5,79,18,...,0,7,6,8,23,0,0,1069,1128,0
904,CAB285000995,벽산블루밍아파트,36.826201,127.113307,0,8,15,1,10,2,...,0,1,10,430,114,0,0,131,1117,7


In [231]:
data['score'].describe()
# 마커 사이즈를 위한, 마커 크기 구하기
min_score = data['score'].min()
max_score = data['score'].max()

data['for_marker_size'] = 1 + ((data['score'] - min_score) / (max_score - min_score)) * 9
data['for_marker_size'] = data['for_marker_size'].round()
data

count     300.000000
mean      499.360000
std       223.028525
min       224.000000
25%       327.000000
50%       438.500000
75%       591.500000
max      1220.000000
Name: score, dtype: float64

,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,sum,score,cluster,for_marker_size
856,CAB285000936,성정중학교,36.830708,127.137961,1,66,1,1,25,27,...,5,10,4,47,0,0,1145,1220,0,10.0
174,ASB288001967,불당상업지구입구,36.811821,127.108607,1,27,11,0,270,36,...,2,7,0,3,1,0,1201,1217,3,10.0
855,CAB285000935,두정양돈농협,36.833573,127.138038,1,54,9,3,70,19,...,5,7,7,28,0,0,1085,1144,0,9.0
924,CAB285001024,두정도서관,36.833743,127.135533,1,49,13,5,79,18,...,7,6,8,23,0,0,1069,1128,0,9.0
904,CAB285000995,벽산블루밍아파트,36.826201,127.113307,0,8,15,1,10,2,...,1,10,430,114,0,0,131,1117,7,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,CAB285071775,쌍용역(방아다리공원방면),36.793798,127.120947,0,21,25,2,43,28,...,4,14,10,28,0,0,160,230,4,1.0
2060,CAB285071558,현대6차아파트,36.791416,127.124305,1,19,23,3,75,15,...,3,12,10,36,0,0,153,227,4,1.0
180,ASB288010485,Y시티아파트,36.797603,127.108500,1,13,2,1,7,17,...,1,6,0,9,0,0,210,227,3,1.0
1970,CAB285071457,제5산단,36.740141,127.255961,0,1,0,0,0,0,...,2,2,83,51,0,1,1,225,8,1.0


### pivot 테이블 만들기

In [143]:
# 각 클러스터마다 집계하기

pivot_table = sorted_score_df.pivot_table(index=['cluster', '정류장번호'])

# Sort the pivot table by 'cluster' in ascending order and 'score' in descending order
sorted_pivot_table = pivot_table.sort_values(by=['cluster', 'score'], ascending=[True, False])
sorted_pivot_table

# 각 클러스터마다, score 점수가 높은 10개의 버스 정류장을 선별
top_10_by_cluster = sorted_pivot_table.groupby('cluster', group_keys=False).apply(lambda group: group.nlargest(10, 'score'))
top_10_by_cluster

bus_id_list= []
for idx in range(len(top_10_by_cluster)):
    data = top_10_by_cluster.index[idx][1]
    bus_id_list.append(data)

bus_id_list

K_df= sorted_score_df[sorted_score_df['정류장번호'].isin(bus_id_list)]
K_df


,정류장번호,정류장명,위도,경도,MT1,CS2,PS3,SC4,AC5,PK6,...,PM9,cluster,공원거리구분_A,공원거리구분_B,CH거리구분_A,CH거리구분_B,하천거리구분_A,하천거리구분_B,sum,score
856,CAB285000936,성정중학교,36.830708,127.137961,1,66,1,1,25,27,...,0,0,5,10,4,47,0,0,1145,1220
174,ASB288001967,불당상업지구입구,36.811821,127.108607,1,27,11,0,270,36,...,0,10,2,7,0,3,1,0,1201,1217
857,CAB285000937,성정중학교,36.829957,127.137722,2,61,1,1,16,25,...,0,0,5,9,3,60,0,0,1098,1183
855,CAB285000935,두정양돈농협,36.833573,127.138038,1,54,9,3,70,19,...,0,0,5,7,7,28,0,0,1085,1144
924,CAB285001024,두정도서관,36.833743,127.135533,1,49,13,5,79,18,...,0,0,7,6,8,23,0,0,1069,1128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,CAB285001115,광덕3리(은골),36.678280,127.061827,0,0,0,0,0,0,...,0,15,0,0,1,0,0,0,6,8
1605,CAB285001982,해수동입구,36.674110,127.038501,0,0,0,0,0,1,...,0,15,0,0,0,0,0,0,8,8
1606,CAB285001983,해수동입구,36.673966,127.038438,0,0,0,0,0,1,...,0,15,0,0,0,0,0,0,8,8
990,CAB285001116,광덕리,36.679835,127.065977,0,0,0,0,0,0,...,0,15,0,0,0,1,0,0,6,7


In [34]:
# import pandas as pd
# import folium



# # Load your DataFrame (ctc_df)
# # ctc_df = pd.read_csv('your_data.csv')
# ctc_df = sorted_score_df.head(188)

# # Convert latitude and longitude columns to float if needed
# ctc_df['위도'] = ctc_df['위도'].astype(float)
# ctc_df['경도'] = ctc_df['경도'].astype(float)

# # Calculate the mean latitude and longitude for centering the map
# center_lat = ctc_df['위도'].mean()
# center_lon = ctc_df['경도'].mean()

# # Create a Folium map centered at the mean location
# m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# # Add markers for each latitude and longitude pair in your DataFrame
# for index, row in ctc_df.iterrows():
#     folium.Marker(
#         location=[row['위도'], row['경도']],
#         popup=row['정류장명'],  # Optional popup information
#         icon=folium.Icon(color='blue')  # Optional icon customization
#     ).add_to(m)

# # Save the map to an HTML file
# path = '/Users/iseunghyeon/Desktop/프로젝트/data/folium시각화/{when}.html'.format(when=when)
# m.save(path)


## plotly

In [232]:
import plotly.express as px


df = data.copy()
# Create a scatter map using Plotly Express
fig = px.scatter_mapbox(
    df,
    lat='위도',
    lon='경도',
    color='cluster',  # Color points based on the cluster column
    size_max=20,      # Adjust point size
    zoom=9,           # Initial map zoom level
    hover_name='정류장명',
    size='for_marker_size'
)

# Customize the map layout
# https://plotly.com/python/mapbox-layers/
map_style = ["open-street-map", "carto-positron", "carto-darkmatter", "stamen-terrain", "stamen-toner" ,"stamen-watercolor" ]
fig.update_layout(mapbox_style=map_style[0])  # Choose a map style
fig.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},  # Adjust margin
    mapbox=dict(center=dict(lat=36.83682, lon=127.2077)),  # Set initial center
)

# Show the plot
fig.show()


In [233]:
# 시각화 파일 다운로드
import plotly.io as pio
now = datetime.now()

when = now.strftime('%Y-%m-%d %H:%M:%S')

# Save the map to an HTML file
path = '/Users/iseunghyeon/Desktop/프로젝트/data/천안시_proj_data/plotly시각화/{when}.html'.format(when=when)
# fig.save(path)

pio.write_html(fig,path)


## folium

In [38]:
import pandas as pd
import folium
from datetime import datetime

now = datetime.now()

when = now.strftime('%Y-%m-%d %H:%M:%S')
when

# Load your DataFrame (ctc_df)
# ctc_df = pd.read_csv('your_data.csv')
ctc_df = sorted_score_df.head(188)

# Assuming you have the 'outer' DataFrame with latitude and longitude columns
# outer = pd.read_csv('outer_data.csv')

# Convert latitude and longitude columns to float if needed
ctc_df['위도'] = ctc_df['위도'].astype(float)
ctc_df['경도'] = ctc_df['경도'].astype(float)
outer['위도'] = outer['위도'].astype(float)
outer['경도'] = outer['경도'].astype(float)

# Calculate the mean latitude and longitude for centering the map
center_lat = (ctc_df['위도'].mean() + outer['위도'].mean()) / 2
center_lon = (ctc_df['경도'].mean() + outer['경도'].mean()) / 2

# Create a Folium map centered at the mean location
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers for ctc_df with blue color
for index, row in outer.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['정류장명'],  # Optional popup information
        icon=folium.Icon(color='red')  # Blue icon
    ).add_to(m)

# Add markers for outer with red color
for index, row in ctc_df.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['정류장명'],  # Optional popup information
        icon=folium.Icon(color='blue')  # Red icon
    ).add_to(m)

# Save the map to an HTML file
path = '/Users/iseunghyeon/Desktop/프로젝트/data/folium시각화/{when}.html'.format(when=when)
m.save(path)
